# Advent Of Code - 2019

## --- Day 3: Crossed Wires ---
The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
````

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

```
R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135
````

What is the Manhattan distance from the central port to the closest intersection?

In [2]:
def load(filename):
    return[ l.split(',') for l in open(filename)]


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return '{}:{}'.format(self.x, self.y)
        
class Wire:
    DIRS = { 'R': (1, 0), 'D': (0, -1), 'L': (-1, 0), 'U': (0, 1) }

    def __init__(self, wire):
        self.x = 0
        self.y = 0
        self.dx = 0
        self.dy = 0
        self.instr = []
        self.path = [Point(0, 0)]
        for instr in wire:
            self.move(instr)

    def move(self, instr):
        d, n = instr[0], int(instr[1:])
        dx, dy = self.DIRS[d]
        self.x += dx * n
        self.y += dy * n
        self.instr.append(instr)
        self.path.append(Point(self.x, self.y))

    def lines(self):
        a = self.path[0]
        for x in self.path[1:]:
            b = x
            p = a, x
            a = x
            yield p


def manhattan(a, b):
    xa, ya = a.x, a.y
    xb, yb = b.x, b.y
    return abs(xa - xb) + abs(ya - yb)


def norm(a, b):
    return manhattan(a, b)


def cmp(a, b):
    """ compare function """
    return (a > b) - (a < b)
    

def is_between(c, a, b):
    """ is point c inside segmeent ab """
    return ((b.x - a.x) * (c.y - a.y) == (c.x - a.x) * (b.y - a.y) and 
            abs(cmp(a.x, c.x) + cmp(b.x, c.x)) <= 1 and
            abs(cmp(a.y, c.y) + cmp(b.y, c.y)) <= 1)


def intersect(a, b, c, d):
    """ Segement Intersection
    """
    
    # a1x + b1y = c1
    a1 = b.y - a.y
    b1 = a.x - b.x
    c1 = a1 * (a.x) + b1 * (a.y)

    # a2x + b2y = c2
    a2 = d.y - c.y
    b2 = c.x - d.x
    c2 = a2 * (c.x) + b2 * (c.y)

    # determinant
    det = a1 * b2 - a2 * b1

    # parallel line
    if det == 0:
        return None

    # intersect point(x,y)
    x = ((b2 * c1) - (b1 * c2)) / det
    y = ((a1 * c2) - (a2 * c1)) / det
    
    i = Point(int(x), int(y))
    if is_between(i, a, b) and is_between(i, c, d):
        return i
    
    return None
    

In [3]:
wires = load("03a.txt")

# wire path
w1 = Wire(wires[0])
w2 = Wire(wires[1])

# segments
s1 = [p for p in w1.lines()]
s2 = [p for p in w2.lines()]

## intersections
o = Point(0, 0)
min_distance = None
min_intersection = None
pp = None
for x in s1:
    for y in s2:
        a, b = x
        c, d = y
        p = intersect(a, b, c, d)
        if p:
            n = manhattan(p, o)
            if not min_distance or min_distance > n:
                min_distance = n
                min_intersection = p

print(min_distance, min_intersection)

1431 -343:1088


## --- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

- R75,D30,R83,U83,L12,D49,R71,U7,L72
- U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
- R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
- U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?

In [4]:
wires = load("03b.txt")

w1 = Wire(wires[0])
w2 = Wire(wires[1])

o = Point(0, 0)
min_steps = None
min_intersection = None
steps_1 = 0
steps_2 = 0
pp = None
for x in s1:
    steps_2 = 0
    a, b = x

    for y in s2:
        c, d = y
        p = intersect(a, b, c, d)
        if p:
            
            i_steps_1 = steps_1 + norm(a, p)
            i_steps_2 = steps_2 + norm(c, p)
            
            n = i_steps_1 + i_steps_2
            
            if not min_steps or min_steps > n:
                min_steps = n
                min_intersection = p
        
        steps_2 += norm(c, d)
        
    steps_1 += norm(a, b)

print(min_steps, min_intersection)


48012 -1806:494
